In [1]:
from ortools.linear_solver import pywraplp
import os
import pandas as pd
import numpy as np
import slim_python as slim

In [3]:
cd ..

C:\Users\danie\Documents\StageDaniel


In [15]:
def load_data(name='breastcancer'):
# requirements for CSV data file
# - outcome variable in first column
# - outcome variable values should be [-1, 1] or [0, 1]
# - first row contains names for the outcome variable + input variables
# - no empty cells
    data_name = name
    data_dir = os.getcwd() + '/data/'
    data_csv_file = data_dir + data_name + '_processed.csv'

    # load data file from csv
    df = pd.read_csv(data_csv_file, sep = ',')
    data = df.as_matrix()
    data_headers = list(df.columns.values)
    N = data.shape[0]

    # setup Y vector and Y_name
    Y_col_idx = [0]
    Y = data[:, Y_col_idx]
    Y_name = [data_headers[j] for j in Y_col_idx]
    Y[Y == 0] = -1

    # setup X and X_names
    X_col_idx = [j for j in range(data.shape[1]) if j not in Y_col_idx]
    X = data[:, X_col_idx]
    X_names = [data_headers[j] for j in X_col_idx]

    # insert a column of ones to X for the intercept
    X = np.insert(arr = X, obj = 0, values = np.ones(N), axis = 1)
    X_names.insert(0, '(Intercept)')

    # run sanity checks
    slim.check_data(X = X, Y = Y, X_names = X_names)
    
    return (X, X_names, Y, Y_name)

In [17]:
(X, X_names, Y, Y_name) = load_data('example_scoring')

C:\Users\danie\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]


In [18]:
    coef_constraints = slim.SLIMCoefficientConstraints(variable_names = X_names, ub = 5, lb = -5)
    #choose upper and lower bounds for the intercept coefficient
    #to ensure that there will be no regularization due to the intercept, choose
    #
    #intercept_ub < min_i(min_score_i)
    #intercept_lb > max_i(max_score_i)
    #
    #where min_score_i = min((Y*X) * \rho) for rho in \Lset
    #where max_score_i = max((Y*X) * \rho) for rho in \Lset
    #
    #setting intercept_ub and intercept_lb in this way ensures that we can always
    # classify every point as positive and negative
    scores_at_ub = (Y * X) * coef_constraints.ub
    scores_at_lb = (Y * X) * coef_constraints.lb
    non_intercept_ind = np.array([n != '(Intercept)' for n in X_names])
    scores_at_ub = scores_at_ub[:, non_intercept_ind]
    scores_at_lb = scores_at_lb[:, non_intercept_ind]
    max_scores = np.fmax(scores_at_ub, scores_at_lb)
    min_scores = np.fmin(scores_at_ub, scores_at_lb)
    max_scores = np.sum(max_scores, 1)
    min_scores = np.sum(min_scores, 1)

    intercept_ub = -min(min_scores) + 1
    intercept_lb = -max(max_scores) + 1
    coef_constraints.set_field('ub', '(Intercept)', intercept_ub)
    coef_constraints.set_field('lb', '(Intercept)', intercept_lb)
    coef_constraints.view()
    
    slim_input = {
        'X': X,
        'X_names': X_names,
        'Y': Y,
        'C_0': 0.01,
        'w_pos': 1.0,
        'w_neg': 1.0,
        'L0_min': 0,
        'L0_max': float('inf'),
        'err_min': 0,
        'err_max': 1.0,
        'pos_err_min': 0,
        'pos_err_max': 1.0,
        'neg_err_min': 0,
        'neg_err_max': 1.0,
        'coef_constraints': coef_constraints
    }


+---------------+-------+------+--------+-------+------+
| variable_name | vtype | sign |   lb   |   ub  | C_0j |
+---------------+-------+------+--------+-------+------+
|  (Intercept)  |   I   | nan  | -124.0 | 126.0 | 0.0  |
|       x1      |   I   | nan  |  -5.0  |  5.0  | nan  |
|       x2      |   I   | nan  |  -5.0  |  5.0  | nan  |
+---------------+-------+------+--------+-------+------+


In [19]:
from slim_python.helper_functions import *
from slim_python.SLIMCoefficientConstraints import SLIMCoefficientConstraints
from math import ceil, floor
input = slim_input
print_flag = False


"""
:param input: dictionary with the following keys
%Y          N x 1 np.array of labels (-1 or 1 only)
%X          N x P np.matrix of feature values (should include a column of 1s to act as an intercept
%X_names    P x 1 list of strings with names of the feature values (all unique and Intercept name)

:return:
%slim_IP
%slim_info
"""

#check preconditions
assert 'X' in input, 'no field named X  in input'
assert 'X_names' in input, 'no field named X_names in input'
assert 'Y' in input, 'no field named Y in input'
assert input['X'].shape[0] == input['Y'].shape[0]
assert input['X'].shape[1] == len(input['X_names'])
assert all((input['Y'] == 1) | (input['Y'] == -1))

XY = input['X'] * input['Y']

#sizes
N = input['X'].shape[0]
P = input['X'].shape[1]
pos_ind = np.flatnonzero(input['Y'] == 1)
neg_ind = np.flatnonzero(input['Y'] == -1)
N_pos = len(pos_ind)
N_neg = len(neg_ind)
binary_data_flag = np.all((input['X'] == 0) | (input['X'] == 1))

#outcome variable name
if ('Y_name' in input) and (type(input['Y_name']) is list):
    input['Y_name'] = input['Y_name'][0]
elif ('Y_name' in input) and (type(input['Y_name']) is str):
    pass
else:
    input['Y_name'] = 'Outcome'

#TODO: check intercept conditions
## first column of X should be all 1s
## first element of X_name should be '(Intercept)'

#set default parameters
input = get_or_set_default(input, 'C_0', 0.01, print_flag = print_flag)
input = get_or_set_default(input, 'w_pos', 1.0, print_flag = print_flag)
input = get_or_set_default(input, 'w_neg', 2.0 - input['w_pos'], print_flag = print_flag)
input = get_or_set_default(input, 'L0_min', 0, print_flag = print_flag)
input = get_or_set_default(input, 'L0_max', P, print_flag = print_flag)
input = get_or_set_default(input, 'err_min', 0.00, print_flag = print_flag)
input = get_or_set_default(input, 'err_max', 1.00, print_flag = print_flag)
input = get_or_set_default(input, 'pos_err_min', 0.00, print_flag = print_flag)
input = get_or_set_default(input, 'pos_err_max', 1.00, print_flag = print_flag)
input = get_or_set_default(input, 'neg_err_min', 0.00, print_flag = print_flag)
input = get_or_set_default(input, 'neg_err_max', 1.00, print_flag = print_flag)

#internal parameters
input = get_or_set_default(input, 'C_1', float('nan'), print_flag = print_flag)
input = get_or_set_default(input, 'M', float('nan'), print_flag = print_flag)
input = get_or_set_default(input, 'epsilon', 0.001, print_flag = print_flag)

#coefficient constraints
if 'coef_constraints' in input:
    coef_constraints = input['coef_constraints']
else:
    coef_constraints = SLIMCoefficientConstraints(variable_names = input['X_names'])


assert len(coef_constraints) == P

# bounds
rho_lb = np.array(coef_constraints.lb)
rho_ub = np.array(coef_constraints.ub)
rho_max = np.maximum(np.abs(rho_lb), np.abs(rho_ub))
beta_ub = rho_max
beta_lb = np.zeros_like(rho_max)
beta_lb[rho_lb > 0] = rho_lb[rho_lb > 0]
beta_lb[rho_ub < 0] = rho_ub[rho_ub < 0]

# signs
signs = coef_constraints.sign
sign_pos = signs == 1
sign_neg = signs == -1

#types
types = coef_constraints.get_field_as_list('vtype')
rho_type = ''.join(types)
#TODO: add support for custom variable types

#class-based weights
w_pos = input['w_pos']
w_neg = input['w_neg']
w_total = w_pos + w_neg
w_pos = 2.0 * (w_pos/w_total)
w_neg = 2.0 * (w_neg/w_total)
assert w_pos > 0.0
assert w_neg > 0.0
assert w_pos + w_neg == 2.0

#L0 regularization penalty
C_0j = np.copy(coef_constraints.C_0j)
L0_reg_ind = np.isnan(C_0j)
C_0j[L0_reg_ind] = input['C_0']
C_0 = C_0j
assert(all(C_0[L0_reg_ind] > 0.0))

#L1 regularization penalty
L1_reg_ind = L0_reg_ind
if not np.isnan(input['C_1']):
    C_1 = input['C_1']
else:
    C_1 = 0.5 * min(w_pos/N, w_neg/N, min(C_0[L1_reg_ind] / np.sum(rho_max)))
C_1 = C_1 * np.ones(shape = (P,))
C_1[~L1_reg_ind] = 0.0
assert(all(C_1[L1_reg_ind] > 0.0))

# model size bounds
L0_min = max(input['L0_min'], 0.0)
L0_max = min(input['L0_max'], np.sum(L0_reg_ind))
L0_min = ceil(L0_min)
L0_max = floor(L0_max)
assert(L0_min <= L0_max)

# total positive error bounds
pos_err_min = 0.0 if np.isnan(input['pos_err_min']) else input['pos_err_min']
pos_err_max = 1.0 if np.isnan(input['pos_err_max']) else input['pos_err_max']
pos_err_min = max(ceil(N_pos*pos_err_min), 0)
pos_err_max = min(floor(N_pos*pos_err_max), N_pos)

# total negative error bounds
neg_err_min = 0.0 if np.isnan(input['neg_err_min']) else input['neg_err_min']
neg_err_max = 1.0 if np.isnan(input['neg_err_max']) else input['neg_err_max']
neg_err_min = max(ceil(N_neg*neg_err_min), 0)
neg_err_max = min(floor(N_neg*neg_err_max), N_neg)

# total error bounds
err_min = 0.0 if np.isnan(input['err_min']) else input['err_min']
err_max = 1.0 if np.isnan(input['err_max']) else input['err_max']
err_min = max(ceil(N*err_min), 0)
err_max = min(floor(N*err_max), N)

# sanity checks for error bounds
assert(err_min <= err_max)
assert(pos_err_min <= pos_err_max)
assert(neg_err_min <= neg_err_max)
assert(err_min >= 0)
assert(pos_err_min >= 0)
assert(neg_err_min >= 0)
assert(err_max <= N)
assert(pos_err_max <= N_pos)
assert(neg_err_max <= N_neg)

#TODO: strengthen bounds
#loss constraint parameters
epsilon  = input['epsilon']
if np.isnan(input['M']):
    max_points = np.maximum(XY * rho_lb, XY * rho_ub)
    max_score_reg = np.sum(-np.sort(-max_points[:, L0_reg_ind])[:, 0:int(L0_max)], axis = 1)
    max_score_no_reg = np.sum(max_points[:, ~L0_reg_ind], axis = 1)
    max_score = max_score_reg + max_score_no_reg
    M = max_score + 1.05 * epsilon
else:
    M = input['M']

#sanity checks for loss constraint parameters
M = M * np.ones(shape = (N,))
M_max = max(np.sum(abs(XY) * rho_max, axis = 1)) + 1.05 * input['epsilon']
assert(len(M) == N)
assert(all(M > 0))
assert(all(M <= M_max))
assert(epsilon > 0.0)
assert(epsilon < 1.0)



In [20]:
M

array([149.00105, 191.00105, 139.00105, 161.00105, 139.00105, 166.00105,
       124.00105, 251.00105, 154.00105, 154.00105])

In [50]:
XY

array([[-1, -1, -4],
       [ 1,  4,  9],
       [-1, -1, -2],
       [ 1,  4,  3],
       [-1, -2, -1],
       [ 1,  5,  3],
       [-1,  0,  0],
       [ 1, 20,  5],
       [-1, -1, -5],
       [-1, -1, -5]], dtype=int64)

In [41]:
#### CREATE ORTools IP

#TODO: describe IP

# x = [loss_pos, loss_neg, rho_j, alpha_j]

#optional constraints:
# objval = w_pos * loss_pos + w_neg * loss_min + sum(C_0j * alpha_j) (required for callback)
# L0_norm = sum(alpha_j) (required for callback)

#rho = P x 1 vector of coefficient values
#alpha  = P x 1 vector of L0-norm variables, alpha(j) = 1 if lambda_j != 0
#beta   = P x 1 vector of L1-norm variables, beta(j) = abs(lambda_j)
#error  = N x 1 vector of loss variables, error(i) = 1 if error on X(i)
#pos_err = auxiliary variable := sum(error[i]) for i: y_i = +1
#neg_err = auxiliary variable := sum(error[i]) for i: y_i = +1
#l0_norm = auxiliary variable := L0_norm = sum(alpha[j])

## IP VARIABLES

#objective costs (we solve min total_error + N * C_0 * L0_norm + N
err_cost = np.ones(shape = (N,))
err_cost[pos_ind] = w_pos
err_cost[neg_ind] = w_neg
C_0 = N * C_0
C_1 = N * C_1

#variable-related values
obj = [0.0] * P + C_0.tolist() + C_1.tolist() + err_cost.tolist()
ub = rho_ub.tolist() + [1] * P + beta_ub.tolist() + [1] * N
lb = rho_lb.tolist() + [0] * P + beta_lb.tolist() + [0] * N
ctype  = rho_type + 'B'*P + 'C'*P + 'B'*N

#variable-related names
rho_names   = ['rho_' + str(j) for j in range(0, P)]
alpha_names = ['alpha_' + str(j) for j in range(0, P)]
beta_names = ['beta_' + str(j) for j in range(0, P)]
error_names = ['error_' + str(i) for i in range(0, N)]
var_names = rho_names + alpha_names + beta_names + error_names

#variable-related error checking
n_var = 3 * P + N
assert(len(obj) == n_var)
assert(len(ub) == n_var)
assert(len(lb) == n_var)
assert(len(ctype) == n_var)
assert(len(var_names) == n_var)

#add variables
slim_IP = cplex.Cplex()
slim_IP.objective.set_sense(slim_IP.objective.sense.minimize)
slim_IP.variables.add(obj = obj, lb = lb, ub = ub, types = ctype, names=var_names)

#Loss Constraints
#Enforce z_i = 1 if incorrect classification)
#M_i * z_i >= XY[i,].dot(rho) + epsilon
for i in range(0, N):
    slim_IP.linear_constraints.add(names = ["error_" + str(i)],
                                   lin_expr = [cplex.SparsePair(ind = rho_names + [error_names[i]],
                                                                val = XY[i,].tolist() + [M[i]])],
                                   senses = "G",
                                   rhs = [epsilon])

# 0-Norm LB Constraints:
# lambda_j,lb * alpha_j <= lambda_j <= Inf
# 0 <= lambda_j - lambda_j,lb * alpha_j < Inf
for j in range(0, P):
    slim_IP.linear_constraints.add(names = ["L0_norm_lb_" + str(j)],
                                   lin_expr = [cplex.SparsePair(ind = [rho_names[j], alpha_names[j]],
                                                                val = [1.0, -rho_lb[j]])],
                                   senses = "G",
                                   rhs = [0.0])

# 0-Norm UB Constraints:
# lambda_j <= lambda_j,ub * alpha_j
# 0 <= -lambda_j + lambda_j,ub * alpha_j
for j in range(0, P):
    slim_IP.linear_constraints.add(names = ["L0_norm_ub_" + str(j)],
                                   lin_expr = [cplex.SparsePair(ind = [rho_names[j], alpha_names[j]],
                                                                val = [-1.0, rho_ub[j]])],
                                   senses = "G",
                                   rhs = [0.0])


# 1-Norm Positive Constraints:
#actual constraint: lambda_j <= beta_j
#cplex constraint:  0 <= -lambda_j + beta_j <= Inf
for j in range(0, P):
    slim_IP.linear_constraints.add(names = ["L1_norm_pos_" + str(j)],
                                   lin_expr = [cplex.SparsePair(ind = [rho_names[j], beta_names[j]],
                                                                val = [-1.0, 1.0])],
                                   senses = "G",
                                   rhs = [0.0])


# 1-Norm Negative Constraints:
#actual constraint: -lambda_j <= beta_j
#cplex constraint:  0 <= lambda_j + beta_j <= Inf
for j in range(0, P):
    slim_IP.linear_constraints.add(names = ["L1_norm_neg_" + str(j)],
                                   lin_expr = [cplex.SparsePair(ind = [rho_names[j], beta_names[j]],
                                                                val = [1.0, 1.0])],
                                   senses = "G",
                                   rhs = [0.0])

# flags for whether or not we will add contraints
add_L0_norm_constraint = (L0_min > 0) or (L0_max < P)
add_total_error_constraint = (err_min > 0) or (err_max < N)
add_pos_error_constraint = (pos_err_min > 0) or (pos_err_max < N_pos) or (add_total_error_constraint)
add_neg_error_constraint = (neg_err_min > 0) or (neg_err_max < N_neg) or (add_total_error_constraint)

### auxiliary variables and bounds
total_l0_norm_name = ['total_l0_norm']
total_error_name = ['total_error']
total_error_pos_name = ['total_error_pos_name']
total_error_neg_name = ['total_error_neg_name']

slim_IP.variables.add(names = total_l0_norm_name,
                      obj = [0.0],
                      lb = [L0_min],
                      ub = [L0_max],
                      types = 'I')

slim_IP.variables.add(names = total_error_name,
                      obj = [0.0],
                      lb = [err_min],
                      ub = [err_max],
                      types = 'I')

slim_IP.variables.add(names = total_error_pos_name,
                      obj = [0.0],
                      lb = [pos_err_min],
                      ub = [pos_err_max],
                      types = 'I')

slim_IP.variables.add(names = total_error_neg_name,
                      obj = [0.0],
                      lb = [neg_err_min],
                      ub = [neg_err_max],
                      types = 'I')


# L0_norm constraint
#if add_L0_norm_constraint:
slim_IP.linear_constraints.add(names = ["total_L0_norm"],
                               lin_expr = [cplex.SparsePair(ind = alpha_names + total_l0_norm_name,
                                                            val = [-1.0] * P + [1.0])],
                               senses = "E",
                               rhs = [0.0])

# total_pos_error variable definition constraint
#err_pos = sum(error[i]) for i in pos_ind
#if add_pos_error_constraint:
slim_IP.linear_constraints.add(names = ["total_pos_error"],
                               lin_expr = [cplex.SparsePair(ind = [error_names[i] for i in pos_ind] + total_error_pos_name,
                                                            val = [-1.0] * N_pos + [1.0])],
                               senses = "E",
                               rhs = [0.0])


# total_neg_error variable definition constraint
#err_neg = sum(error[i]) for i in neg_ind
#if add_neg_error_constraint:
slim_IP.linear_constraints.add(names = ["total_neg_error"],
                               lin_expr = [cplex.SparsePair(ind = [error_names[i] for i in neg_ind] + total_error_neg_name,
                                                            val = [-1.0] * N_neg + [1.0])],
                               senses = "E",
                               rhs = [0.0])

# total_error variable definition constraint
#if add_total_error_constraint:
slim_IP.linear_constraints.add(names = ["total_error"],
                               lin_expr = [cplex.SparsePair(ind = total_error_name + total_error_pos_name + total_error_neg_name,
                                                            val = [1.0, -1.0, -1.0])],
                               senses = "E",
                               rhs = [0.0])


#### Drop Variables and Constraints
variables_to_drop = []
constraints_to_drop = []

# drop alpha[j] and beta[j] if L0_reg_ind == False
no_L0_reg_ind = np.flatnonzero(~L0_reg_ind).tolist()
variables_to_drop += ["alpha_" + str(j) for j in no_L0_reg_ind]
variables_to_drop += ["beta_" + str(j) for j in no_L0_reg_ind]
constraints_to_drop += ["L0_norm_lb_" + str(j) for j in no_L0_reg_ind]
constraints_to_drop += ["L0_norm_ub_" + str(j) for j in no_L0_reg_ind]
constraints_to_drop += ["L1_norm_pos_" + str(j) for j in no_L0_reg_ind]
constraints_to_drop += ["L1_norm_neg_" + str(j) for j in no_L0_reg_ind]

# drop beta[j] if L0_reg_ind == False or L1_reg_ind == False
no_L1_reg_ind = np.flatnonzero(~L1_reg_ind).tolist()
variables_to_drop += ["beta_" + str(j) for j in no_L1_reg_ind]
constraints_to_drop += ["L1_norm_pos_" + str(j) for j in no_L1_reg_ind]
constraints_to_drop += ["L1_norm_neg_" + str(j) for j in no_L1_reg_ind]

# drop alpha[j] and beta[j] if values are fixed at 0
fixed_value_ind = np.flatnonzero(rho_lb == rho_ub).tolist()
variables_to_drop += ["alpha_" + str(j) for j in fixed_value_ind]
variables_to_drop += ["beta_" + str(j) for j in fixed_value_ind]
constraints_to_drop += ["L0_norm_lb_" + str(j) for j in fixed_value_ind]
constraints_to_drop += ["L0_norm_ub_" + str(j) for j in fixed_value_ind]
constraints_to_drop += ["L1_norm_pos_" + str(j) for j in fixed_value_ind]
constraints_to_drop += ["L1_norm_neg_" + str(j) for j in fixed_value_ind]

# drop constraints based on signs
sign_pos_ind = np.flatnonzero(sign_pos).tolist()
sign_neg_ind = np.flatnonzero(sign_neg).tolist()

# drop L1_norm_neg constraint for any variable rho[j] >= 0
constraints_to_drop += ["L1_norm_neg_" + str(j) for j in sign_pos_ind]

# drop L0_norm_lb constraint for any variable rho[j] >= 0
constraints_to_drop += ["L0_norm_lb_" + str(j) for j in sign_pos_ind]

# drop L1_norm_pos constraint for any variable rho[j] <= 0
constraints_to_drop += ["L1_norm_pos_" + str(j) for j in sign_neg_ind]

# drop L0_norm_ub constraint for any variable rho[j] <= 0
constraints_to_drop += ["L0_norm_ub_" + str(j) for j in sign_neg_ind]

if len(constraints_to_drop) > 0:
    constraints_to_drop = list(set(constraints_to_drop))
    slim_IP.linear_constraints.delete(constraints_to_drop)

if len(variables_to_drop) > 0:
    variables_to_drop = list(set(variables_to_drop))
    slim_IP.variables.delete(variables_to_drop)

#create info dictionary for debugging
rho_names = [n for n in rho_names if n not in variables_to_drop]
alpha_names = [n for n in alpha_names if n not in variables_to_drop]
beta_names = [n for n in beta_names if n not in variables_to_drop]

slim_info = {
    #
    # key parameters
    #
    "C_0": C_0,
    "C_1": C_1,
    "w_pos": w_pos,
    "w_neg": w_neg,
    "err_min": err_min,
    "err_max": err_max,
    "pos_err_min": pos_err_min,
    "pos_err_max": pos_err_max,
    "neg_err_min": neg_err_min,
    "neg_err_max": neg_err_max,
    "L0_min": L0_min,
    "L0_max": L0_max,
    "N": N,
    "P": P,
    "N_pos": N_pos,
    "N_neg": N_neg,
    "rho_ub": rho_ub,
    "rho_lb": rho_lb,
    "M": M,
    "epsilon": epsilon,
    #
    "binary_data_flag": binary_data_flag,
    "pos_ind": pos_ind,
    "neg_ind": neg_ind,
    "L0_reg_ind": L0_reg_ind,
    "L1_reg_ind": L1_reg_ind,
    #
    "n_variables": slim_IP.variables.get_num(),
    "n_constraints": slim_IP.linear_constraints.get_num(),
    "names": slim_IP.variables.get_names(),
    #
    # MIP variables indices
    #
    "rho_idx":  slim_IP.variables.get_indices(rho_names),
    "alpha_idx": slim_IP.variables.get_indices(alpha_names),
    "beta_idx": slim_IP.variables.get_indices(beta_names),
    "error_idx":  slim_IP.variables.get_indices(error_names),
    "total_l0_norm_idx": slim_IP.variables.get_indices(total_l0_norm_name),
    "total_error_idx": slim_IP.variables.get_indices(total_error_name),
    "total_error_pos_idx": slim_IP.variables.get_indices(total_error_pos_name),
    "total_error_neg_idx": slim_IP.variables.get_indices(total_error_neg_name),
    #
    # MIP variables names
    #
    "rho_names": rho_names,
    "alpha_names": alpha_names,
    "beta_names": beta_names,
    "rho_names": rho_names,
    "error_names": error_names,
    "total_error_name": total_error_name,
    "total_error_pos_name": total_error_pos_name,
    "total_error_neg_name": total_error_neg_name,
    #
    "X_names": input['X_names'],
    "Y_name": input['Y_name'],
    #
    # dropped
    "variables_to_drop": variables_to_drop,
    "constraints_to_drop": constraints_to_drop,
}




In [42]:
    # setup SLIM IP parameters
    # see docs/usrccplex.pdf for more about these parameters
    slim_IP.parameters.timelimit.set(200.0) #set runtime here
    #TODO: add these default settings to create_slim_IP
    slim_IP.parameters.randomseed.set(0)
    slim_IP.parameters.threads.set(1)
    slim_IP.parameters.parallel.set(1)
    slim_IP.parameters.output.clonelog.set(0)
    slim_IP.parameters.mip.tolerances.mipgap.set(np.finfo(np.float).eps)
    slim_IP.parameters.mip.tolerances.absmipgap.set(np.finfo(np.float).eps)
    slim_IP.parameters.mip.tolerances.integrality.set(np.finfo(np.float).eps)
    slim_IP.parameters.emphasis.mip.set(1)


    # solve SLIM IP
    slim_IP.solve()

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Parallel                                1
CPXPARAM_RandomSeed                              0
CPXPARAM_Emphasis_MIP                            1
CPXPARAM_TimeLimit                               200
CPXPARAM_MIP_Tolerances_AbsMIPGap                2.2204460492503131e-16
CPXPARAM_MIP_Tolerances_MIPGap                   2.2204460492503131e-16
CPXPARAM_MIP_Tolerances_Integrality              2.2204460492503131e-16
Found incumbent of value 6.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 2 times.
MIP Presolve modified 20 coefficients.
Aggregator did 4 substitutions.
Reduced MIP has 18 rows, 17 columns, and 54 nonzeros.
Reduced MIP has 12 binaries, 3 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.07 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve 

In [43]:
    # run quick and dirty tests to make sure that IP output is correct
    slim.check_slim_IP_output(slim_IP, slim_info, X, Y, coef_constraints)

In [44]:
slim_IP.solution.get_values()

[-3.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0]

In [45]:
#### CHECK RESULTS ####
slim_results = slim.get_slim_summary(slim_IP, slim_info, X, Y)
# print(slim_results)

# print model
print(slim_results['string_model'])

# print coefficient vector
print(slim_results['rho'])


+-------------------------------+------------------+-----------+
| PREDICT O IF SCORE >= -3      |                  |           |
| ============================= | ================ | ========= |
| x1                            |         1 points |   + ..... |
| ============================= | ================ | ========= |
| ADD POINTS FROM ROWS 1 to 1   |            SCORE |   = ..... |
+-------------------------------+------------------+-----------+
[-3.  1.  0.]


In [46]:
slim_results

{'solution_status_code': 101,
 'solution_status': 'integer optimal solution',
 'objective_value': 1.0036764705882353,
 'optimality_gap': 1.0036764705882353,
 'objval_lowerbound': 0.0,
 'simplex_iterations': 30,
 'nodes_processed': 0,
 'nodes_remaining': 0,
 'rho': array([-3.,  1.,  0.]),
 'pretty_model': <prettytable.PrettyTable at 0x2b8cca9b648>,
 'string_model': '+-------------------------------+------------------+-----------+\n| PREDICT O IF SCORE >= -3      |                  |           |\n| ============================= | ================ | ========= |\n| x1                            |         1 points |   + ..... |\n| ============================= | ================ | ========= |\n| ADD POINTS FROM ROWS 1 to 1   |            SCORE |   = ..... |\n+-------------------------------+------------------+-----------+',
 'true_positives': 4,
 'true_negatives': 6,
 'false_positives': 0,
 'false_negatives': 0,
 'mistakes': 0,
 'error_rate': 0.0,
 'true_positive_rate': 1.0,
 'false_positiv